In [115]:
import imutils
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.object_detection import non_max_suppression
import numpy as np
import time
import cv2
import PIL
import IPython
from IPython.display import clear_output,display
import pytesseract
#Time Lib
import datetime
import pymysql


try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO

In [116]:
#Connect to SQL
db = pymysql.connect(
  host="localhost",
  user="root",
  passwd="root",
  db="test_connector"  
)
cursor = db.cursor()
print(db)

In [117]:
# USE This method instaed of cv2.imageShow
def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(IPython.display.Image(data=f.getvalue()))

In [118]:
# argvideo = ''
argseast = 'frozen_east_text_detection.pb'
args_min_confid = 0.70
# multiply
pi = np.pi
argspad = 1/pi

In [119]:
def decode_predictions(scores, geometry):
    # grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the
        # geometrical data used to derive potential bounding box
        # coordinates that surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # loop over the number of columns
        for x in range(0, numCols):
            # if our score does not have sufficient probability,
            # ignore it
            if scoresData[x] < args_min_confid:
                continue

            # compute the offset factor as our resulting feature
            # maps will be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            # extract the rotation angle for the prediction and
            # then compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            # use the geometry volume to derive the width and height
            # of the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            # compute both the starting and ending (x, y)-coordinates
            # for the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            # add the bounding box coordinates and probability score
            # to our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    # return a tuple of the bounding boxes and associated confidences
    return (rects, confidences)

In [120]:
# second can be used to derive the bounding box coordinates of text
layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(argseast)

[INFO] loading EAST text detector...


In [121]:
import asyncio
import time
import IPython
from IPython.display import display,clear_output

In [122]:
def tesseract_read_roi(roi):
    path_ = 'images/snapshot/';pathArray=[];txtArray=[];remark=[];
    if roi == []: 
        display('');
    else :    
        length = len(roi)-1;
        while length >= 0:
            img = roi[length]

            # process img for ocr : resizeX2 -> Gray Scale -> mode treshold
            img = cv2.resize(img,None,fx=2,fy=2)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

            # Threshold
            # adaptive gaussian threshold
        #     img_agt = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,9,2)
            # adaptive mean threshold
        #     img_amt = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,9,2)
            # OTSU threshold
            ret_otsu,img_otsu = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            # Otsu's thresholding after Gaussian filtering
        #     blur = cv2.GaussianBlur(img,(5,5),0)
        #     ret_otsug,img_otsug = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

             # in order to apply Tesseract v4 to OCR text we must supply
            # (1) a language, (2) an OEM flag of 4, indicating that the we
            # wish to use the LSTM neural net model for OCR, and finally
            # (3) an OEM value, in this case, 7 which implies that we are
            # treating the ROI as a single line of text

            config = ("-l eng --oem 1 --psm 7")
            text = pytesseract.image_to_string(img_otsu, config=config)
            
            # display
            showarray(img_otsu);
            print(text);
            
            # Write into database
            ts = datetime.datetime.now().timestamp()
            ts = str(int(ts))
            path = path_+ts+'.jpg'
            cv2.imwrite(path,img_otsu)
            pathArray.append(path)
            path = path_
            
            #text
            txtArray.append(text)
            
            length -= 1
            
        #end while loop
        timestamp = datetime.datetime.now();
        pathlist = pathArray;
        txtlist = txtArray;
        remark = '';
        cursor.execute(
            """
            INSERT into testRecord (Timestamp, ImgPath, Text, Remark) values(%s,%r,%r,%s)
            """,(timestamp,pathlist,txtlist,remark))
#         db.commit()

In [123]:
async def async_counter_snap(asyncState):
    while 1:
        await asyncio.sleep(1);
        asyncState.counter += 1;

In [124]:
async def async_run_video(asyncState):
    try:    
        while 1:
            await asyncio.sleep(0.00000000000000001);
            clear_output(wait=True);
            roi = [];
            # grab the current frame, then handle if we are using a
            # VideoStream or VideoCapture object
            vs = asyncState.vs
            frame = vs.read()

            # check to see if we have reached the end of the stream
            if frame is None:
                break

            # resize the frame, maintaining the aspect ratio
            frame = imutils.resize(frame, width=1000)
            orig = frame.copy();roi_crop = frame.copy();
            orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)

            # if our frame dimensions are None, we still need to compute the
            # ratio of old frame dimensions to new frame dimensions
            if asyncState.W is None or asyncState.H is None:
                (asyncState.H, asyncState.W) = frame.shape[:2]
                asyncState.rW = asyncState.W / float(asyncState.newW)
                asyncState.rH = asyncState.H / float(asyncState.newH)

            # resize the frame, this time ignoring aspect ratio
            frame = cv2.resize(frame, (asyncState.newW, asyncState.newH))

            # construct a blob from the frame and then perform a forward pass
            # of the model to obtain the two output layer sets
            blob = cv2.dnn.blobFromImage(frame, 1.0, (asyncState.newW, asyncState.newH),(123.68, 116.78, 103.94), swapRB=True, crop=False)
            net.setInput(blob)
            (scores, geometry) = net.forward(layerNames)

            # decode the predictions, then  apply non-maxima suppression to
            # suppress weak, overlapping bounding boxes
            (rects, confidences) = decode_predictions(scores, geometry)
            boxes = non_max_suppression(np.array(rects), probs=confidences)
            
            
            # loop over the bounding boxes
            for (startX, startY, endX, endY) in boxes:
                # scale the bounding box coordinates based on the respective
                # ratios
                # in order to obtain a better OCR of the text we can potentially
                # apply a bit of padding surrounding the bounding box -- here we
                # are computing the deltas in both the x and y directions
                dX = int((endX - startX) * argspad)
                dY = int((endY - startY) * argspad)
                startX = int((startX * asyncState.rW)-dX/5)
                startY = int((startY * asyncState.rH)-dY/5)
                endX = int((endX * asyncState.rW)+dX/5)
                endY = int((endY * asyncState.rH)+dY/5)
                tmp = roi_crop[startY:endY, startX:endX] 
                roi.append(tmp);
                # draw the bounding box on the frame
                cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)

            # update the FPS counter
            fps = asyncState.fps
            fps.update()

            # show the output frame
            showarray(orig);
            display(asyncState.counter);
            if asyncState.counter >= 5:
                asyncState.counter=0;
                asyncState.roi = roi;
                tesseract_read_roi(asyncState.roi);
                continue;
            # Display the frame until new frame is available

    except KeyboardInterrupt:
            print('\n\nKeyboard exception received. Exiting.')    
            # stop the timer and display FPS information
            fps.stop();vs.stop()
            print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
            print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))          
        

In [125]:
async def main():
    # Setup an object to track the state
    asyncState = type('', (), {})()
    asyncState.counter = 0;
    asyncState.marker = '';
    asyncState.roi = []; asyncState.magik_num = 1/np.pi;
    
    # if a video path was not supplied, grab the reference to the web cam
    print("[INFO] starting video stream...")
    asyncState.vs = VideoStream(src=0).start()

    # start the FPS throughput estimator
    asyncState.fps = FPS().start()
    # initialize the original frame dimensions, new frame dimensions,
    # and ratio between the dimensions
    asyncState.argsw = 320; asyncState.argsh = 320
    (asyncState.W, asyncState.H) = (None, None)
    (asyncState.newW, asyncState.newH) = (asyncState.argsw, asyncState.argsh)
    (asyncState.rW, asyncState.rH) = (None, None)

    await asyncio.gather(async_counter_snap(asyncState),async_run_video(asyncState))

In [126]:
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
asyncio.get_event_loop()

<_UnixSelectorEventLoop running=False closed=False debug=False>

In [127]:
try:
    future = loop.create_task(main())
    loop.run_until_complete(future)
except:
    pass
finally:
    print('close loop')
    loop.close();



Keyboard exception received. Exiting.
[INFO] elasped time: 22.83
[INFO] approx. FPS: 2.54
close loop


In [128]:
loop

<_UnixSelectorEventLoop running=False closed=True debug=False>

In [129]:
#Show Records
cursor.execute(
    """
    SELECT * FROM testRecord
    """)

for x in cursor:
  print(x)

(1, datetime.datetime(2019, 3, 28, 14, 36, 43), 'Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(2, datetime.datetime(2019, 3, 28, 14, 42, 12), 'Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(3, datetime.datetime(2019, 3, 28, 14, 42, 44), 'Desktop/python/jupyter/txt-e.jpg,Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(4, datetime.datetime(2019, 3, 28, 8, 22, 53), "('Desktop/python/jupyter/txt-x.jpg','Desktop/python/jupyter/txt-y.jpg','Desktop/python/jupyter/txt-z.jpg')", "('intania90','intania91','intania92')", 'test_pathlist')
(11, datetime.datetime(2019, 3, 28, 16, 54, 50), "'images/snapshot/1553766890.jpg'", "'txtArray'", '[]')
(12, datetime.datetime(2019, 3, 28, 16, 54, 50), "('images/snapshot/1553766890.jpg','images/snapshot/1553766890.jpg')", "('txtArray','txtArray')", '[]')
(13, datetime.datetime(2019, 3, 28, 17, 9, 31), "('images/snapshot/1553767770.jpg')", "('ein cued')", '')
(14, datetime.datetime(2019, 3, 28, 17, 9, 37), "('images/snapshot/155

In [130]:
db.commit()

In [59]:
path = 'images/snapshot/1553765174.jpg'
image = cv2.imread(path)

In [60]:
ts = datetime.datetime.now().timestamp()
ts = str(int(ts))
path_ = 'images/snapshot/'
pathl = path_+ts+'.jpg'
cv2.imwrite(path,image)
# pathArray.append(path)
# #text
# print(text);
# txtArray.append(text)
# length -= 1

#end while loop
timestamp = datetime.datetime.now();
pathlist = pathl;
txtlist = 'txtArray';
remark = '[]';
cursor.execute(
    """
    INSERT into testRecord (Timestamp, ImgPath, Text, Remark) values(%s,%r,%r,%s)
    """,(timestamp,pathlist,txtlist,remark))
# db.commit()

1

In [61]:
timestamp = datetime.datetime.now();
pathArray = []; txtArray = []
pathArray.append(pathl);txtArray.append(txtlist);
pathArray.append(pathl);txtArray.append(txtlist);
remark = '[]';
cursor.execute(
    """
    INSERT into testRecord (Timestamp, ImgPath, Text, Remark) values(%s,%r,%r,%s)
    """,(timestamp,pathArray,txtArray,remark))

1

In [63]:
#Show Records
cursor.execute(
    """
    SELECT * FROM testRecord
    """)

for x in cursor:
  print(x)

(1, datetime.datetime(2019, 3, 28, 14, 36, 43), 'Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(2, datetime.datetime(2019, 3, 28, 14, 42, 12), 'Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(3, datetime.datetime(2019, 3, 28, 14, 42, 44), 'Desktop/python/jupyter/txt-e.jpg,Desktop/python/jupyter/txt-e.jpg', 'intania93', 'test4')
(4, datetime.datetime(2019, 3, 28, 8, 22, 53), "('Desktop/python/jupyter/txt-x.jpg','Desktop/python/jupyter/txt-y.jpg','Desktop/python/jupyter/txt-z.jpg')", "('intania90','intania91','intania92')", 'test_pathlist')
(11, datetime.datetime(2019, 3, 28, 16, 54, 50), "'images/snapshot/1553766890.jpg'", "'txtArray'", '[]')
(12, datetime.datetime(2019, 3, 28, 16, 54, 50), "('images/snapshot/1553766890.jpg','images/snapshot/1553766890.jpg')", "('txtArray','txtArray')", '[]')
